In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import uproot
import vector

In [54]:
# rootf = "/home/llr/cms/kalipoliti/C10630p1_miniAOD/src/HeavyIonsAnalysis/Configuration/test/HiForestMiniAOD_2events.root"
# rootf = "/home/llr/cms/kalipoliti/C10630p1_miniAOD/src/HeavyIonsAnalysis/Configuration/test/HiForestMiniAOD_10000events.root"
rootf = "/home/llr/cms/kalipoliti/C10630p1_miniAOD/src/HeavyIonsAnalysis/Configuration/test/HiForestMiniAOD.root"

gen_particles = pd.DataFrame()
gen_particles_cols = ["pt", "eta", "phi", "pdg", "sta"]

tracks = pd.DataFrame()
tracks_cols = ["trkJetId", "trkPt", "trkEta", "trkPhi"]

with uproot.open(rootf) as fin:
    for col in gen_particles_cols:
        branch = fin[f"bDecayAna/hi/{col}"].array(library="pd")
        gen_particles[col] = branch
        
    for col in tracks_cols:
        branch = fin[f"ak4PFJetAnalyzer/t/{col}"].array(library="pd")
        tracks[col] = branch

In [55]:
## Add cuts to remove some of the useless particles
# min_pt = 0.8
# gen_particles = gen_particles[gen_particles["pt"] > min_pt]
gen_particles = gen_particles.reset_index()
gen_particles = gen_particles.rename(columns={"subentry": "id"})
gen_particles

,entry,id,pt,eta,phi,pdg,sta
0,0,0,39.031250,0.446243,-1.932383,-13,1
1,0,1,0.438232,0.329998,-0.998757,211,100
2,0,2,0.282959,0.299674,-1.555025,-211,100
3,0,3,0.446045,-0.449614,0.185459,-211,100
4,0,4,1.073242,0.279538,0.129017,321,100
...,...,...,...,...,...,...,...
219235,999,436,1.956055,4.252506,-1.067518,-321,1
219236,999,437,0.191772,3.861970,-2.046646,211,1
219237,999,438,0.394775,4.162710,-1.166153,211,1
219238,999,439,0.223633,3.845847,-0.139651,-211,1


In [56]:
tracks = tracks.reset_index()
tracks = tracks.rename(columns={"subentry": "id"})
tracks

,entry,id,trkJetId,trkPt,trkEta,trkPhi
0,0,0,0,4.085938,1.483749,-2.272626
1,0,1,0,3.847656,1.554247,-2.415013
2,0,2,0,3.818359,1.472213,-2.133852
3,0,3,0,4.503906,1.470199,-2.308178
4,0,4,0,1.137695,1.537217,-2.295294
...,...,...,...,...,...,...
20173,999,33,8,3.976562,-1.891537,-0.356556
20174,999,34,9,1.011719,2.867153,-1.332258
20175,999,35,9,1.778320,2.748131,-0.954039
20176,999,36,11,2.074219,-1.027436,-0.139941


In [53]:
matched_ids_noCuts = []
matched_ids_medCuts = []

for track_row in tracks.itertuples():
    entry_gen_particles = gen_particles[gen_particles["entry"] == track_row.entry]
    min_dr2 = 1.
    min_dr2_medCuts = 1.
    
    matched_id_noCuts = -1
    matched_id_medCuts = -1
    
    for gen_row in entry_gen_particles.itertuples():
        track_vec = vector.obj(rho=0, phi=track_row.trkPhi, eta=track_row.trkEta)
        gen_vec = vector.obj(rho=0, phi=gen_row.phi, eta=gen_row.eta)
        
        dr = track_vec.deltaR(gen_vec)
        dr2 = dr * dr
        if dr2 < min_dr2:
            min_dr2 = dr2
            matched_id = gen_row.id
            
        rel_pt = track_row.trkPt / gen_row.pt
        if (dr2 < 0.0004) and  (rel_pt > 0.8) and (rel_pt < 1.2):
            if (dr2 < min_dr2_medCuts):
                min_dr2_medCuts = dr2
                matched_id_medCuts = gen_row.id
    matched_ids_noCuts.append(matched_id_noCuts)
    matched_ids_medCuts.append(matched_id_medCuts)
    
tracks["matchId"] = matched_ids_noCuts
tracks["matchIdMedCuts"] = matched_ids_medCuts
tracks

KeyboardInterrupt: 

In [6]:
def fun(x, a):
    return x*x + a

In [34]:
d = {"id": [0, 1, 2, 3, 4], "x": [0, 1, 2, 3, 4], "y": [5, 6, 7, 8, 9], "z": [10, 20, 30, 40, 50]}
df = pd.DataFrame(d)
# new_col = df.apply(fun, axis=1)
track_v = vector.obj(rho=0, phi=2, eta=8.5)
df["dr2"] = df.apply(lambda row: (vector.obj(rho=0, phi=row.x, eta=row.y).deltaR(track_v))**2, axis=1)

track_pt = 28
df["rel_pt"] = df.apply(lambda row: row.z / track_pt, axis=1)
df = df[(df["rel_pt"] > 0.8) & (df["rel_pt"] < 1.2)]
df = df.sort_values(["dr2"])
id = df["id"].to_numpy()[0]
print(id)
df

2


,id,x,y,z,dr2,rel_pt
2,2,2,7,30,2.25,1.071429


In [57]:
matched_ids_noCuts = []
matched_ids_medCuts = []

for track_row in tracks.itertuples():
    entry_gen_particles = gen_particles[gen_particles["entry"] == track_row.entry].copy()
    min_dr2 = 1.
    min_dr2_medCuts = 1.
    
    matched_id_noCuts = -1
    matched_id_medCuts = -1

    track_v = vector.obj(rho=0, phi=track_row.trkPhi, eta=track_row.trkEta)
    track_pt = track_row.trkPt

    entry_gen_particles["dr2"] = entry_gen_particles.apply(lambda row: (vector.obj(rho=0, phi=row.phi, eta=row.eta).deltaR(track_v))**2, axis=1)
    entry_gen_particles["rel_pt"] = entry_gen_particles.apply(lambda row: row.pt / track_pt, axis=1)

    entry_gen_particles = entry_gen_particles.sort_values(by=["dr2"], ascending=True)
    matched_id_noCuts = entry_gen_particles["id"].to_numpy()[0]

    sel_rel_pt = (entry_gen_particles["rel_pt"] > 0.8) & (entry_gen_particles["rel_pt"] < 1.2)
    # print(entry_gen_particles[sel_rel_pt])
    sel_rel_pt_ids = entry_gen_particles.loc[sel_rel_pt, "id"].to_numpy()
    if (len(sel_rel_pt_ids) > 0):
        matched_id_medCuts = sel_rel_pt_ids[0]

    matched_ids_noCuts.append(matched_id_noCuts)
    matched_ids_medCuts.append(matched_id_medCuts)

tracks["matchId"] = matched_ids_noCuts
tracks["matchIdMedCuts"] = matched_ids_medCuts
tracks
    

KeyboardInterrupt: 

In [45]:
sel = tracks["matchId"] != tracks["matchIdMedCuts"]
sel &= tracks["matchIdMedCuts"] != -1
tracks[sel]

,entry,id,trkJetId,trkPt,trkEta,trkPhi,matchId,matchIdMedCuts
12,0,12,4,2.275391,2.179205,0.273255,161,159
16,0,16,5,1.297852,2.075564,1.972138,316,271
53,1,26,8,1.150391,-0.077456,1.191825,275,266


In [49]:
gen_particles[(gen_particles["entry"] == 0) & (gen_particles["id"] == 159)]

,entry,id,pt,eta,phi,pdg,sta,dr2,rel_pt
159,0,159,2.236328,1.661526,0.716921,-211,1,100.0,0.0
